## <span style="color:#000066"> Question 10: Simulated Dataset </span> 

In this exercise, we will generate simulated data, and will then use this data to perform best subset selection.<br>

Assumption: Reader has basic understanding of python and statistics.

*Suggestion: This tutorial should be read in conjunction with Chapter6 of ISLR*

#### Importing all dependencies from module

In [6]:
from chapter6_imports import *

#### Simulated Dataset:
Let's Generate a data set with p = 20 features, n = 1,000 observations, and an associated quantitative response vector generated according to the model Y = bX+e

In [13]:
X = np.random.normal(size=(1000, 20))
beta = np.random.normal(size=20)
beta[3] = 0
beta[5] = 0
beta[9] = 0
e = np.random.normal(size=1000)
y = np.dot(X, beta) + e
X = pd.DataFrame(X, columns = ['col'+str(i) for i in range(0,20)])

In [14]:
def get_combinations(arr):
    var_list = []
    for r in range(1, len(arr)+1):
        variables = list(itertools.combinations(arr, r)) 
        var_list.append(variables)
    return list(itertools.chain(*var_list))

def fit_linear_model(X, Y):
    lin_model = LinearRegression()
    lin_model.fit(X,Y)
    RSS = ((lin_model.predict(X)- Y) ** 2).sum()
    R_squared = lin_model.score(X,Y)
    return lin_model, RSS, R_squared

def get_selection_metrices(model_df):
    m = len(model_data['target'])
    p = len(model_data['data'].columns)
    model_df['numb_features'] = model_df.index.str.split('_').str[1]
    sigma_hat_sqd = (1/(m - p -1)) * min(model_df['RSS'])
    model_df['C_p'] = (1/m) * (model_df['RSS'] + model_df['numb_features'].astype(float).multiply(sigma_hat_sqd *2 ))
    model_df['AIC'] = (1/(m*sigma_hat_sqd)) * (model_df['RSS'] + 2 * model_df['numb_features'].astype(float) * sigma_hat_sqd )
    model_df['BIC'] = (1/(m*sigma_hat_sqd)) * (model_df['RSS'] +  np.log(m) * model_df['numb_features'].astype(float) * sigma_hat_sqd )
    adj_factor = np.divide((m-1), (m-model_df['numb_features'].astype(int)-1))
    model_df['R_sqd_adj'] = 1 - ((1 - model_df['R_squared'])*adj_factor)
    return model_df

def select_model(criteria, data):
    if criteria == 'AIC':
        return data.loc[data['AIC'].idxmin()]['model']
    if criteria == 'BIC':
        return data.loc[data['BIC'].idxmin()]['model']
    if criteria == 'R_sqd_adj':
        return data.loc[data['R_sqd_adj'].idxmax()]['model']

def best_subset_algorithm(model_data, criteria):
    var_list = get_combinations(model_data["data"].columns)
    least_rss_model = {"M_"+str(var_num):{"model":None,"RSS":np.inf, "R_squared":np.inf} 
                       for var_num in range(1, len(model_data['data'].columns)+1)}
    for var in var_list:
        var_num = len(var)
        X_data = model_data['data'][list(var)]
        lin_model, RSS, R_sqd = fit_linear_model(X_data, model_data['target'])
        if RSS < least_rss_model["M_"+str(var_num)]["RSS"]:
            least_rss_model["M_"+str(var_num)]["model"] = lin_model
            least_rss_model["M_"+str(var_num)]["RSS"] = RSS
            least_rss_model["M_"+str(var_num)]["R_squared"] = R_sqd
    model_df = pd.DataFrame.from_dict(least_rss_model, orient='index')
    model_df = get_selection_metrices(model_df)
    model_selected = select_model(criteria, model_df)
    if plot_metric:
        plot_metrices(model_df)
    return model_selected, model_df

In [ ]:
model_data = {"data":X, "target":y}
model, df = best_subset_algorithm(model_data, 'AIC')

{'data':          col0      col1      col2      col3      col4      col5      col6  \
 0    0.498562 -1.437112 -1.661133  0.076551 -0.188752  2.834009  0.586856   
 1   -0.930648  0.164994 -1.198215 -1.410123  1.691301  0.330118  0.346337   
 2    0.014053  1.568369  0.574697 -0.437782 -0.121699 -0.307276 -1.312732   
 3    0.036235 -0.922353  1.377586  1.656422  0.601030 -0.482925  1.661287   
 4   -0.078807  0.935513  0.832913 -0.572206  0.169740 -0.597618  0.306949   
 ..        ...       ...       ...       ...       ...       ...       ...   
 995 -1.684951  0.447822 -0.350039 -0.169640 -1.327384 -1.296848 -0.898410   
 996  0.805573  0.671641  0.165863 -1.082898  0.420734  1.004694  0.163571   
 997  0.161341 -0.050943 -0.285367 -0.580748 -1.733872 -1.259416  0.713532   
 998 -0.213935  0.603700 -0.518903 -1.907791  1.255255  0.196503  0.510143   
 999  0.568642  0.699376 -0.225287  1.400769 -1.838780  0.393048  0.997483   
 
          col7      col8      col9     col10     col11